# Extraction d'Embeddings avec ResNet50

Ce notebook montre comment utiliser le modèle pré-entraîné **ResNet50** de PyTorch pour extraire les embeddings d'une image. 

Le script retire la dernière couche fully connected du modèle pour obtenir une représentation vectorielle (embedding) des caractéristiques de l'image. 

⚠️ **Important** : Assurez-vous que le chemin de l'image est correct. Si le chemin contient des espaces, utilisez des guillemets autour du chemin complet.

In [ ]:
# Importation des bibliothèques nécessaires
import torch
from torchvision import models, transforms
from PIL import Image

# Pour afficher correctement les images dans le notebook (si besoin)
%matplotlib inline

## 1. Configuration du Device et Chargement du Modèle

Nous définissons le device global (GPU si disponible, sinon CPU) et chargeons le modèle **ResNet50** pré-entraîné. 
Le modèle est mis en mode évaluation et déplacé sur le device sélectionné. 
Ensuite, nous retirons la dernière couche fully connected pour obtenir les embeddings.

In [ ]:
# Définir le device global (GPU si disponible, sinon CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation du device : {device}")

# Chargement du modèle pré-entraîné ResNet50
model = models.resnet50(pretrained=True)
model.eval()  # Mode évaluation
model.to(device)  # Déplacer le modèle sur le device

# Retirer la dernière couche fully connected pour obtenir les embeddings
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])
feature_extractor.to(device)  # Déplacer l'extracteur sur le device

## 2. Définition du Pipeline de Prétraitement

Nous définissons ici un pipeline de prétraitement qui va :
- Redimensionner l'image à 256 pixels
- Effectuer un crop centré de 224 pixels
- Convertir l'image en tenseur
- Normaliser l'image avec les moyennes et écarts-types utilisés pour l'entraînement du modèle

Ce pipeline permet de préparer l'image pour le modèle ResNet50.

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),  # Conversion en tensor avec valeurs dans [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

## 3. Définition de la Fonction `get_embedding`

La fonction `get_embedding` ouvre une image, applique le prétraitement et extrait les features (embeddings) en passant l'image par le modèle. 
Le résultat est aplati pour obtenir un vecteur 1D par image.

En cas d'erreur lors de l'ouverture de l'image, une exception est levée.

In [ ]:
def get_embedding(image_path):
    """
    Extrait l'embedding d'une image en utilisant un modèle ResNet50 pré-entraîné.
    
    Args:
        image_path (str): Chemin vers l'image.
    
    Returns:
        torch.Tensor: Les features extraites sous forme d'un vecteur plat.
    """
    try:
        img = Image.open(image_path).convert('RGB')
    except Exception as e:
        raise ValueError(f"Erreur lors de l'ouverture de l'image {image_path} : {e}")
    
    # Appliquer le prétraitement
    input_tensor = preprocess(img)
    input_batch = input_tensor.unsqueeze(0)  # Ajoute la dimension batch
    
    # Déplacer le batch sur le device approprié
    input_batch = input_batch.to(device)
    
    with torch.no_grad():
        features = feature_extractor(input_batch)
    
    # Aplatir le résultat pour obtenir un vecteur 1D par image
    features = features.view(features.size(0), -1)
    return features

## 4. Exemple d'Utilisation

Dans cette section, nous utilisons la fonction `get_embedding` sur une image d'exemple. 
Assurez-vous de remplacer le chemin de l'image par un chemin valide sur votre système. 

Pour exécuter ce code en dehors d'un notebook, un script Python peut utiliser la clause `if __name__ == '__main__':`.

In [ ]:
if __name__ == "__main__":
    # Remplacez ce chemin par le chemin réel de votre image
    image_path = r"C:\Users\HP\Desktop\LIPSTIP\Logos_dataset\earlier_003466547.jpg"
    
    embedding = get_embedding(image_path)
    print("Embedding shape:", embedding.shape)